In [ ]:
# Importar e instalar dependencias
!pip install sacremoses
import pandas as pd
import numpy as np
import re
from collections import Counter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 19.1 MB/s eta 0:00:00


# CARGA DE DATOS

In [ ]:
!pip install gdown
# Descarga el archivo del dataset de drive usando gdown
url = 'https://drive.google.com/file/d/1LkEJ3rstkdyhUGWi9O2YKQMe0wC_ZyDd/view?usp=sharing'
file_id = url.split('/')[-2]
!gdown --id $file_id

# Carga el dataset usando pandas
sd = pd.read_csv('/content/Suicide_Detection.csv')

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1LkEJ3rstkdyhUGWi9O2YKQMe0wC_ZyDd
From (redirected): https://drive.google.com/uc?id=1LkEJ3rstkdyhUGWi9O2YKQMe0wC_ZyDd&confirm=t&uuid=8d52a0fb-3ea6-4a70-96e5-fba5be5b65d5
To: /content/Suicide_Detection.csv
100% 167M/167M [00:06<00:00, 24.1MB/s]


In [ ]:
from transformers import pipeline, MarianTokenizer, MarianMTModel
import pandas as pd
from tqdm import tqdm
import torch
!pip install sacremoses

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Sample data for testing purposes
sd_t = sd.sample(n=200, random_state=None)

# Load a specific tokenizer and model
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es", clean_up_tokenization_spaces=True)
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-es")

# Create a translation pipeline with the specified tokenizer and model
translator = pipeline("translation_en_to_es", model=model, tokenizer=tokenizer, device=device)

def translate_batch(texts, batch_size=32):  # Adjust batch_size for your GPU memory
    translations = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        # Tokenize the batch of texts
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)  # Move inputs to the device
        # Generate translations
        translated_batch = model.generate(**inputs)
        # Decode translations
        decoded_batch = tokenizer.batch_decode(translated_batch, skip_special_tokens=True)
        translations.extend(decoded_batch)
    return translations

# Translate the texts in batches
sd_t['spanish_text'] = translate_batch(sd_t['text'].tolist())

# Verify by printing the first few rows
print(sd_t.head())



cuda


100%|██████████| 7/7 [01:19<00:00, 11.39s/it]

        Unnamed: 0                                               text  \
135216      203006  Least deadly drugs to overdose on?My life is o...   
173650      260677  happy trans day of visibility !!!! you bitches...   
50584        75748  I don’t want to die but I don’t have anything ...   
157219      235846  Why I dislike my sister Okay so at my house I ...   
32918        49413  ok ok, i have a deal for you guys so basically...   

              class                                       spanish_text  
135216      suicide  Mi vida ha terminado, tengo autismo, hay un ju...  
173650  non-suicide  feliz trans día de visibilidad!!!! perras son ...  
50584       suicide  No quiero morir, pero no tengo nada por lo que...  
157219  non-suicide  Por qué no me gusta mi hermana De acuerdo, así...  
32918   non-suicide  ok ok, tengo un trato para ustedes chicos así ...  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define la ruta donde quieres guardar el archivo en tu Google Drive
file_path = '/content/drive/MyDrive/Suicide_Detection_translated_5.csv'

# Guarda el DataFrame en un archivo CSV en Google Drive
sd_t.to_csv(file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sd_t.head(20)

,Unnamed: 0,text,class,spanish_text
135216,203006,Least deadly drugs to overdose on?My life is o...,suicide,"Mi vida ha terminado, tengo autismo, hay un ju..."
173650,260677,happy trans day of visibility !!!! you bitches...,non-suicide,feliz trans día de visibilidad!!!! perras son ...
50584,75748,I don’t want to die but I don’t have anything ...,suicide,"No quiero morir, pero no tengo nada por lo que..."
157219,235846,Why I dislike my sister Okay so at my house I ...,non-suicide,"Por qué no me gusta mi hermana De acuerdo, así..."
32918,49413,"ok ok, i have a deal for you guys so basically...",non-suicide,"ok ok, tengo un trato para ustedes chicos así ..."
107347,161311,Can life just hurry up and end?Don't give a sh...,suicide,¿Puede la vida simplemente darse prisa y termi...
145260,218047,Really contemplating...Really contemplating su...,suicide,Realmente contemplando...Realmente contempland...
198449,297762,My top 5 songs by my top 4 favorite artists/ba...,non-suicide,Mis 5 canciones principales de mis 4 artistas/...
108803,163526,Went 6 years without a suicidal thought and no...,suicide,Fui 6 años sin un pensamiento suicida y ahora ...
144483,216885,Day 288 of writing something positive that hap...,non-suicide,Día 288 de escribir algo positivo que me pasó ...
